# User interface: the compressed likelihood

This notebook focuses on the final product, and how we expect most (non-experts) users to use our compressed likelihood. 

By compressed likelihood we mean a likelihood that has already marginalized over nuisance (astro) parameters, and that uses a reduced set of parameters to describe the cosmological model. 

The compressed likelihood doesn't know about redshift bins, or band powers, it doesn't know about mean flux, temperature or redshift or reionization.

Summary:
    - Given an input cosmological model, it computes the parameters describing the linear power spectrum (linP).
    - given a set of linP parameters, interpolates over a look-up table and returns the value of log-likelihood.

In [1]:
%matplotlib inline
import numpy as np
import os
## Set default plot size, as normally its a bit too small
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 120
mpl.rcParams['figure.dpi'] = 120
import fit_pk
import lya_results

### Specify cosmological models

We can specify the cosmological model by hand or from a GenIC parameter file (see other notebooks in this folder).

In [2]:
# specify cosmology
cosmo = fit_pk.get_cosmology(mnu=0.3,ns=0.95, omch2=0.13)
# print relevant information about the cosmology object
fit_pk.print_info(cosmo)

H0 = 67.0 ; Omega_b h^2 = 0.022 ; Omega_c h^2 = 0.13 ; Omega_k = 0.0 ; ommnuh2 = 0.00322 ; T_CMB = 2.7255 ; A_s = 2.1e-09 ; n_s = 0.95


### Compute parameters describing the linear power spectrum around $z_\star=3$, $k_p=0.009$ s/km

In [3]:
z_star=3.0
kp_kms=0.009
linP_params=fit_pk.parameterize_cosmology(cosmo,z_star=z_star,kp_kms=kp_kms)

### Evaluate compressed likelihood

This is the likelihood that has already been marginalized over nuisance parameters, and is of course, experiment specific. At this point it has lost all information about nuisance parameters, or data points. 

In [4]:
def compressed_loglike(linP_params,z_star,kp_kms):
    # log(amplitude) of linear power at z_star, kp_kms
    lnA_p = linP_params['linP_kms'][0]
    # logarithmic slope of linear power at z_star, kp_kms
    n_p = linP_params['linP_kms'][1]
    # logarighmic curvature (running) of linear power at z_star, kp_kms
    alpha_p = linP_params['linP_kms'][2]
    # logarithmic growth rate at z_star, around k = 1 h/Mpc
    f_star=linP_params['f_star']
    # logarithmic derivative of Hubble parameter, with respect to EdS, at z_star 
    g_star=linP_params['g_star']
    
    # here we would interpolate a look-up table or similar
    
    # note that we do not expect to have more than 3 parameters in this look up table
    # since (f_star,g_star,alpha_p) will not be well measured
    
    # for now, we can use a Gaussian approximation on the results of Chabanier et al. (2019, Figure 20)
    DL2=np.exp(lnA_p)*kp_kms**3/(2*np.pi**2)
    delta_chi2=lya_results.gaussian_chi2_Chabanier2019(n_p,DL2)
    
    return -0.5*delta_chi2

In [5]:
loglike = compressed_loglike(linP_params,z_star,kp_kms)

As an example, we will use an approximate likelihood extracted from the ellipses in Figure 20 of Chabanier et al. (2019).